<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Computação em Nuvem III
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Apache Spark;</li>
  <li>Data Wrangling com Spark.</li>
</ol>

---

# **Exercícios**

## 1\. Apache Spark

Replique as atividades do item 2.1 e 2.2 para instalar e configurar um cluster Apache Spark na máquina virtual do Google Colab.

In [35]:
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz

In [36]:
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [37]:
!pip install -q pyspark==3.0.0

In [38]:
import os
import findspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import broadcast
from re import U

In [39]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [40]:
!pip install -q findspark==1.4.2

In [41]:
findspark.init()

In [42]:
spark = SparkSession.builder.master("local[*]").appName("pyspark-notebook").getOrCreate()

## 2\. Data Wrangling

A base de dados presente neste [link](https://www.kaggle.com/datasets/bank-of-england/a-millennium-of-macroeconomic-data) contem dados macroeconômicos sobre o Reino Unido desde o século 13.

**2.1\. Data**

Faça o download dos dados utilizando a máquina virutal do Google Colab com o código abaixo.

In [43]:
!wget -q "https://raw.githubusercontent.com/cluster-apps-on-docker/spark-standalone-cluster-on-docker/master/build/workspace/data/uk-macroeconomic-data.csv" -O "uk-macroeconomic-data.csv"

**2.2. Wrangling**

Processe os dados para que a base de dados final apresente os valores da taxa de desemprego (`Unemployment rate`) e população (`Population (GB+NI)`) estejam ordenados por ano decrescente:

```csv
year,population,unemployment_rate
...,...,...
```

Para isso, utilize:

 - Pandas

In [44]:
uk_data_eco = pd.read_csv('uk-macroeconomic-data.csv')
uk_data_eco.head(5)

,Description,Real GDP of England at market prices,Real GDP of England at factor cost,"Real UK GDP at market prices, geographically-consistent estimate based on post-1922 borders","Real UK GDP at factor cost, geographically-consistent estimate based on post-1922 borders","Index of real UK GDP at factor cost - based on changing political boundaries,",Composite estimate of English and (geographically-consistent) UK real GDP at factor cost,HP-filter of log of real composite estimate of English and UK real GDP at factor cost,"Real UK gross disposable national income at market prices, constant border estimate",Real consumption,...,UK Public sector debt.1,UK Public sector debt.2,Central Government Gross Debt,Central Government Gross Debt.1,Trade deficit,Trade deficit.1,Current account,Current account .1,Current account deficit including estimated non-monetary bullion flows,Current account deficit including estimated non-monetary bullion flows.1
0,Units,"£mn, Chained Volume measure, 2013 prices","£mn, Chained Volume measure, 2013 prices","£mn, Chained Volume measure, 2013 prices","£mn, Chained Volume measure, 2013 prices","GB before 1801, GB+Ireland 1801-1920, GB + Nor...",2013=100,approx. % difference from trend,"£mn, Chained Volume measure, 2013 prices. Nom...","£mn, Chained Volume measure, 2013 prices",...,as a % of nominal GDP: measure 1,as a % of nominal GDP: measure 2,"Financial year end, nominal par value £mn","Financial year end, market value £mn",£mn,as a % of nominal GDP,£mn,as a % of nominal GDP,£mn,as a % of nominal GDP
1,1209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
uk_data_eco = uk_data_eco.loc[1:]
col_int = ['Description', 'Population (GB+NI)', 'Unemployment rate']
uk_data_eco = uk_data_eco[col_int].dropna().sort_values('Description', ascending=False)
uk_data_eco

,Description,Population (GB+NI),Unemployment rate
808,2016,65573,4.90
807,2015,65110,5.38
806,2014,64597,6.18
805,2013,64106,7.61
804,2012,63705,7.97
...,...,...,...
651,1859,24138,3.27
650,1858,23914,5.23
649,1857,23689,3.95
648,1856,23466,3.52


In [46]:
uk_data_eco.columns = ['Year', 'Population', 'Unemployment rate']
uk_data_eco.head(5)

,Year,Population,Unemployment rate
808,2016,65573,4.90
807,2015,65110,5.38
806,2014,64597,6.18
805,2013,64106,7.61
804,2012,63705,7.97


In [47]:
uk_data_eco.to_csv('uk-macroeconomic-data-cleaned.csv', index=False)

 - PySpark

In [48]:
uk_data_eco = spark.read.csv(path='uk-macroeconomic-data.csv',sep=',',header=True)

uk_data_eco.show(n=5)

+-----------+------------------------------------+-----------------------------------+-------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------+------------------------------------------------------------------------------+----------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+--------------------+--------------------+--------------------------+-------------------------------------------------+--------------------+--------------------+---------------------------------------+-------------------------------+---------------------------------+------------------+--------------------+----------+-----------------+---------------------------+------------------------------

In [49]:
uk_data_eco.printSchema()

root
 |-- Description: string (nullable = true)
 |-- Real GDP of England at market prices: string (nullable = true)
 |-- Real GDP of England at factor cost : string (nullable = true)
 |-- Real UK GDP at market prices, geographically-consistent estimate based on post-1922 borders: string (nullable = true)
 |-- Real UK GDP at factor cost, geographically-consistent estimate based on post-1922 borders: string (nullable = true)
 |-- Index of real UK GDP at factor cost - based on changing political boundaries, : string (nullable = true)
 |-- Composite estimate of English and (geographically-consistent) UK real GDP at factor cost: string (nullable = true)
 |-- HP-filter of log of real composite estimate of English and UK real GDP at factor cost: string (nullable = true)
 |-- Real UK gross disposable national income at market prices, constant border estimate: string (nullable = true)
 |-- Real consumption: string (nullable = true)
 |-- Real investment: string (nullable = true)
 |-- Stockbuildi

In [50]:
uk_data_eco = uk_data_eco.select(['Description', 'Population (GB+NI)', 'Unemployment rate'])

In [51]:
uk_data_eco = uk_data_eco.\
    withColumnRenamed('Description', 'Year').\
    withColumnRenamed('Population (GB+NI)', 'Population')

In [52]:
uk_data_eco.show(n=5)

+-----+----------+-----------------+
| Year|Population|Unemployment rate|
+-----+----------+-----------------+
|Units|      000s|                %|
| 1209|      null|             null|
| 1210|      null|             null|
| 1211|      null|             null|
| 1212|      null|             null|
+-----+----------+-----------------+
only showing top 5 rows



In [53]:
uk_data_eco_lista = uk_data_eco.\
  filter(uk_data_eco['Year'] == 'Units')

In [54]:
uk_data_eco = uk_data_eco.join(other=broadcast(uk_data_eco_lista),on=['year'],how='left_anti')
uk_data_eco.show()

+----+----------+-----------------+
|Year|Population|Unemployment rate|
+----+----------+-----------------+
|1209|      null|             null|
|1210|      null|             null|
|1211|      null|             null|
|1212|      null|             null|
|1213|      null|             null|
|1214|      null|             null|
|1215|      null|             null|
|1216|      null|             null|
|1217|      null|             null|
|1218|      null|             null|
|1219|      null|             null|
|1220|      null|             null|
|1221|      null|             null|
|1222|      null|             null|
|1223|      null|             null|
|1224|      null|             null|
|1225|      null|             null|
|1226|      null|             null|
|1227|      null|             null|
|1228|      null|             null|
+----+----------+-----------------+
only showing top 20 rows



In [55]:
uk_data_eco.sort(uk_data_eco.Year.desc()).show()

+----+----------+-----------------+
|Year|Population|Unemployment rate|
+----+----------+-----------------+
|2016|     65573|             4.90|
|2015|     65110|             5.38|
|2014|     64597|             6.18|
|2013|     64106|             7.61|
|2012|     63705|             7.97|
|2011|     63285|             8.11|
|2010|     62759|             7.87|
|2009|     62260|             7.61|
|2008|     61824|             5.69|
|2007|     61319|             5.33|
|2006|     60827|             5.42|
|2005|     60413|             4.83|
|2004|     59950|             4.75|
|2003|     59637|             5.01|
|2002|     59366|             5.19|
|2001|     59113|             5.10|
|2000|     58886|             5.46|
|1999|     58684|             5.98|
|1998|     58475|             6.26|
|1997|     58314|             6.97|
+----+----------+-----------------+
only showing top 20 rows

